<a href="https://colab.research.google.com/github/andik71/eduwork-msib4/blob/main/lbp_project_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
# ======================================================
# KONFIGURASI
# ======================================================
INPUT_FILE = 'lbp_datasets_merged.csv'      # File transaksi mentah yang dihasilkan dari generate_transaksi
OUTPUT_CLEANED = 'lbp_dataset_grouped.csv'    # Output file setelah grouping transaksi
OUTPUT_ENCODED = 'lbp_dataset_encoded.csv'    # Output file untuk binary matrix (one-hot encoding)
DELIMITER_CSV = ','

In [ ]:
# ======================================================
# FUNGSI PREPROCESSING
# ======================================================
def preprocess_transactions(df: pd.DataFrame) -> pd.DataFrame:
    """
    Melakukan cleaning dan grouping pada data transaksi.

    Proses:
      1. Validasi keberadaan kolom 'id_transaksi', 'tgl_transaksi', dan 'id_produk'.
      2. Mengonversi kolom 'tgl_transaksi' ke tipe datetime.
      3. Mengelompokkan transaksi berdasarkan 'id_transaksi' dan 'tgl_transaksi'
         serta menyimpan daftar produk unik (tanpa duplikasi dan diurutkan) pada kolom 'items'.

    Output:
      DataFrame dengan kolom: id_transaksi, tgl_transaksi, items
    """
    # Validasi kolom wajib
    required_columns = ['id_transaksi', 'tgl_transaksi', 'id_produk']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Kolom wajib '{col}' tidak ditemukan")

    # Konversi 'tgl_transaksi' ke datetime jika belum
    if not pd.api.types.is_datetime64_any_dtype(df['tgl_transaksi']):
        df['tgl_transaksi'] = pd.to_datetime(df['tgl_transaksi'], errors='coerce')
        if df['tgl_transaksi'].isnull().any():
            raise ValueError("Terdapat nilai yang tidak bisa dikonversi ke datetime pada kolom 'tgl_transaksi'")

    # Grouping berdasarkan 'id_transaksi' dan 'tgl_transaksi'
    grouped = df.groupby(['id_transaksi', 'tgl_transaksi'])['id_produk'] \
                .agg(lambda x: sorted(list(set(x)))) \
                .reset_index() \
                .rename(columns={'id_produk': 'items'})

    return grouped

def encode_transactions(df: pd.DataFrame) -> pd.DataFrame:
    """
    Mengubah kolom 'items' pada DataFrame transaksi menjadi format binary matrix.
    Menggunakan TransactionEncoder dari mlxtend untuk mengonversi daftar item
    menjadi kolom-kolom boolean.

    Output:
      DataFrame yang menggabungkan metadata (id_transaksi, tgl_transaksi) dengan matrix binary.
    """
    try:
        te = TransactionEncoder()
        te_ary = te.fit_transform(df['items'])
        encoded_df = pd.DataFrame(te_ary, columns=te.columns_)
        # Gabungkan metadata transaksi dengan hasil encoding
        encoded_df = pd.concat([df[['id_transaksi', 'tgl_transaksi']].reset_index(drop=True), encoded_df], axis=1)
        return encoded_df
    except Exception as e:
        raise RuntimeError(f"Gagal melakukan encoding: {str(e)}")

In [ ]:
# ======================================================
# MAIN PROGRAM
# ======================================================
if __name__ == "__main__":
    print("=== Program Preprocessing Data Transaksi ===")
    try:
        # Langkah 1: Membaca data mentah
        print("\n1. Membaca data mentah...")
        raw_df = pd.read_csv(INPUT_FILE, delimiter=DELIMITER_CSV)
        print("Contoh data mentah:")
        print(raw_df.head(2))

        # Langkah 2: Cleaning dan grouping transaksi
        print("\n2. Mengelompokkan transaksi...")
        cleaned_df = preprocess_transactions(raw_df)
        cleaned_df.to_csv(OUTPUT_CLEANED, index=False)
        print(f"Data grouped disimpan ke: {OUTPUT_CLEANED}")
        print("Contoh data grouped:")
        print(cleaned_df.head(2))
        print(f"Total data grouped: {cleaned_df.shape[0]} baris")

        # Langkah 3: Encoding transaksi ke binary matrix
        print("\n3. Mengonversi transaksi ke binary matrix...")
        encoded_df = encode_transactions(cleaned_df)
        encoded_df.to_csv(OUTPUT_ENCODED, index=False, sep=DELIMITER_CSV)
        print(f"Data encoded disimpan ke: {OUTPUT_ENCODED}")
        print("Contoh data encoded (2 transaksi & 5 kolom pertama):")
        print(encoded_df.iloc[:2, :5])
        print(f"Total data encoded: {encoded_df.shape[0]} baris dengan {encoded_df.shape[1]} fitur")

        print("\n=== Preprocessing selesai! ===")
    except Exception as e:
        print(f"\nERROR: {str(e)}")

=== Program Preprocessing Data Transaksi ===

1. Membaca data mentah...
Contoh data mentah:
            id_transaksi        tgl_transaksi id_produk  nama_produk  \
0  LBP202412011637560001  2024-12-01 16:37:56    BQFLBB  Buket Bunga   
1  LBP202412011637560001  2024-12-01 16:37:56    BQSKGA  Buket Snack   

  sub_kategori fitur_produk harga_produk  jumlah_produk  total_transaksi  
0        Biasa   10 Tangkai     Rp100,00              1            10000  
1   Gery Salut      5 Snack      Rp25,00              1             2500  

2. Mengelompokkan transaksi...
Data grouped disimpan ke: lbp_dataset_grouped.csv
Contoh data grouped:
            id_transaksi       tgl_transaksi                             items
0  LBP202408011100400045 2024-08-01 11:00:40  [BQFLEB, BQSKBB, BQSKPA, GCSDWD]
1  LBP202408011102140037 2024-08-01 11:02:14  [BQFLBA, BQSKHA, BQSKPA, BQSKPC]
Total data grouped: 15996 baris

3. Mengonversi transaksi ke binary matrix...
Data encoded disimpan ke: lbp_dataset_encoded.cs